In [1]:
import time
import pathlib
import pandas as pd 
from selenium import webdriver

Path for Selenium Chromedriver

In [21]:
path_chromedriver = pathlib.Path('chromedriver\\chromedriver.exe')

Make DataFrame

In [3]:
df_youtube = pd.DataFrame(columns = ['date_published', 'title', 'views', 'keywords', 'video_id'])

Define function for scraping the following data from Youtube: date, title, views and ID

In [19]:
def youtube_channel(extension, count, df=df_youtube): 
    '''
    Transforming df_youtube DataFrame:
    1) Initialize and run chromedriver;
    2) Find the chanel, open videos tab and scrape date, title, views and ID of "count"-number latest videos;
    3) Assign each data category to each column in DataFrame.
    '''
    
    chrome_driver = path_chromedriver.name
    driver = webdriver.Chrome(chrome_driver)
    youtube = 'https://www.youtube.com/'
    driver.get(f'{youtube}results?search_query={extension}')
    time.sleep(1.5)
    channel_entrance = driver.find_element_by_css_selector('#main-link')
    channel_entrance.click()
    time.sleep(0.5)
    channel_videos_entrance = driver.find_element_by_css_selector('#tabsContent > paper-tab:nth-child(4) > div')
    channel_videos_entrance.click()
    time.sleep(2.5)
    part_one = '/html/body/ytd-app/div/ytd-page-manager/ytd-browse[1]/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[1]/ytd-grid-video-renderer['
    part_two = ']/div[1]/div[1]/div[1]/h3/a'
    date_list, title_list, views_list, id_list  = [], [] , [] ,[]
    
    for r in range(1, count + 1):
        url = part_one + str(r) + part_two
        videos_entrance = driver.find_element_by_xpath(url)
        driver.execute_script("arguments[0].click();", videos_entrance)
        time.sleep(1.5)
        date_list.append(driver.find_element_by_css_selector('#date > yt-formatted-string').text)
        title_list.append(driver.find_element_by_css_selector('#container > h1 > yt-formatted-string').text)
        views_list.append(driver.find_element_by_css_selector('#count > yt-view-count-renderer > span.view-count.style-scope.yt-view-count-renderer').text)
        time.sleep(2.5)
        video_ids = driver.find_elements_by_xpath('//*[@id="page-manager"]/ytd-watch-flexy')
        
        for video in video_ids:
            video_id = str(video.get_attribute('video-id'))
            id_list.append(video_id)
        driver.back()
        
    df['date_published'] = date_list
    df['title'] = title_list
    df['views'] = views_list
    df['video_id'] = id_list
    driver.quit()

Run **youtube_channel()** function to extract date, title, views and ID

In [20]:
youtube_channel(input('Enter your channel: ').replace(' ','+'), int(input('Enter amount of latest videos: ')))
df_youtube

Enter your channel: skirmantas malinauskas
Enter amount of latest videos: 5


,date_published,title,views,keywords,video_id
0,7 Mar 2021,Kaip pavogti milijoną?,"67,588 views",NaN,AdUiTzUNrd8
1,28 Feb 2021,Apie tai kalbėti draudžiama,"83,054 views",NaN,Aq-bpR6vFig
2,21 Feb 2021,Safari Kaune ir antstoliai prie Šviniaus durų,"126,101 views",NaN,2l6-mS6pt0c
3,16 Feb 2021,"""CityBee"" mums pamelavo - 110 tūkstančių žmoni...","147,489 views",NaN,wbKznbxe5r4
4,14 Feb 2021,Sudužusios svajonės,"102,142 views",NaN,DyXAs7fDyus
